## 🧪 Pruebas CRUD en Supabase

Este notebook verifica las operaciones CRUD (Crear, Leer, Actualizar, Eliminar) en una tabla de prueba en Supabase.

**Objetivo:** Validar que las credenciales (`SUPABASE_URL` y `SUPABASE_SERVICE_KEY`) son correctas y que podemos realizar operaciones básicas en la base de datos.

### ⚠️ Paso 1: Crear la Tabla de Prueba en Supabase

La librería cliente de Supabase no puede crear tablas. Debes hacerlo desde el editor SQL en tu panel de Supabase.

**Copia y ejecuta el siguiente código SQL en tu editor de Supabase una sola vez:**

```sql
-- Crea la tabla para las pruebas CRUD
CREATE TABLE IF NOT EXISTS test_items (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    created_at TIMESTAMPTZ DEFAULT NOW()
);

-- Asegúrate de que el rol 'service_role' (usado por este notebook) tenga acceso total
ALTER TABLE test_items ENABLE ROW LEVEL SECURITY;
DROP POLICY IF EXISTS "Allow full access to service role" ON test_items;
CREATE POLICY "Allow full access to service role" ON test_items FOR ALL
TO service_role
USING (true)
WITH CHECK (true);
```

Una vez creada, puedes continuar con las celdas de este notebook.

In [ ]:
import os
import sys
from dotenv import load_dotenv
from supabase import create_client, Client

# Cargar variables de entorno desde la raíz del proyecto
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
dotenv_path = os.path.join(project_root, '.env')
load_dotenv(dotenv_path=dotenv_path)

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")

if not supabase_url or not supabase_key:
    print("❌ Faltan las variables SUPABASE_URL o SUPABASE_SERVICE_KEY.")
else:
    print(f"✅ Variables de entorno cargadas. Conectando a: {supabase_url[:25]}...")

In [ ]:
supabase_client: Client | None = None
try:
    supabase_client: Client = create_client(supabase_url, supabase_key)
    print("✅ Cliente de Supabase creado.")
except Exception as e:
    print(f"❌ Error al crear el cliente de Supabase: {e}")

### 2. Crear (Insertar) un Item de Prueba

In [ ]:
import uuid

# Usaremos un nombre único para nuestro item de prueba
test_item_name = f"Test Item {uuid.uuid4()}"
item_data = {"name": test_item_name}

print(f"📋 Datos de prueba listos: {item_data}")

In [ ]:
inserted_id = None
if supabase_client:
    try:
        # Insertar los datos en la tabla 'test_items'
        data, count = supabase_client.table('test_items').insert(item_data).execute()
        
        inserted_record = data[1][0]
        inserted_id = inserted_record['id']
        
        print(f"✅ Item insertado correctamente con ID: {inserted_id}")
    except Exception as e:
        print(f"❌ Error al insertar el item: {e}")
        print("---")
        print("POSIBLES CAUSAS:")
        print("1. ¿Ejecutaste el script SQL del Paso 1 para crear la tabla 'test_items'?")
        print("2. ¿Son correctas las credenciales en tu archivo .env?")
else:
    print("⏩ No se pudo crear el cliente de Supabase. Revisa los errores en celdas anteriores.")

### 3. Leer (Seleccionar) el Item

In [ ]:
if inserted_id and supabase_client:
    try:
        data, count = supabase_client.table('test_items').select('*').eq('id', inserted_id).execute()
        
        if data[1]:
            retrieved_record = data[1][0]
            print("✅ Item encontrado en la base de datos:")
            print(f"- ID: {retrieved_record['id']}")
            print(f"- Nombre: {retrieved_record['name']}")
            assert retrieved_record['name'] == test_item_name
        else:
            print(f"❌ No se encontró el item con ID {inserted_id} que acabamos de insertar.")
    except Exception as e:
        print(f"❌ Error al leer el item: {e}")
else:
    print("⏩ Se omite la lectura porque no se pudo insertar el item previamente.")

### 4. Eliminar el Item de Prueba

In [ ]:
if inserted_id and supabase_client:
    try:
        data, count = supabase_client.table('test_items').delete().eq('id', inserted_id).execute()
        
        if data[1]:
            print(f"✅ Item con ID {inserted_id} eliminado correctamente.")
        else:
            print(f"❌ No se pudo eliminar el item con ID {inserted_id}.")
    except Exception as e:
        print(f"❌ Error al eliminar el item: {e}")
else:
    print("⏩ Se omite la eliminación porque no se pudo insertar el item previamente.")